### Emnist Letters CNN

In [2]:
# import statements

from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import TensorDataset, DataLoader


In [3]:
# Load in and store the data as a Pandas dataframe

filename = "emnist-letters-train.csv"
data = pd.read_csv(filename)

raw_train_data = data.sample(frac=0.9, random_state=1)
raw_test_data = data.drop(raw_train_data.index)

In [24]:
# Now, we need to format the data correctly. Currently, the first column corresponds to the labels 
# and the remaining columns correspond to the pixels of the image in sequential order. 

def preprocess_data(data):
    X = data.iloc[:, 1:].values / 255
    y = data.iloc[:, 0].values - 1 
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.long)

X_train, y_train = preprocess_data(raw_train_data)
X_test, y_test = preprocess_data(raw_test_data)

# We will also create a dataloader to make the training process easier. 

train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

test_data = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [25]:
# Let's build out the neural network now. I will use the PyTorch framework to set up a simple CNN. 

class CharNet(nn.Module):
    def __init__(self):
        super(CharNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2,2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2,2)

        self.fc1 = nn.Linear(7*7*64, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 26)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))

        x = x.view(-1, 7*7*64)
        x = self.fc2(self.relu3(self.fc1(x)))
        probs = F.log_softmax(x, dim=1)

        return probs    

In [31]:
# With the model architecture in place, we can begin training!

classifier = CharNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=0.0001)

classifier.train()
epochs = 20

for epoch in range(epochs):
    train_loss = 0
    for batch, (X, y) in enumerate(train_loader):
        X = X.view(-1, 1, 28, 28)
        optimizer.zero_grad()
        output = classifier(X)
        loss = loss_function(output, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss /= len(train_loader)
    print(f"Epoch: {epoch}, Train Loss: {train_loss}")

torch.save(classifier.state_dict(), 'model_weights2.pth')

# model_weights: 91.64 test accuracy
# model_weights2: 

Epoch: 0, Train Loss: 1.2888335960926869
Epoch: 1, Train Loss: 0.6090573491670305
Epoch: 2, Train Loss: 0.4468435456482671
Epoch: 3, Train Loss: 0.372079268563166
Epoch: 4, Train Loss: 0.3318261778633818
Epoch: 5, Train Loss: 0.30139616050655316
Epoch: 6, Train Loss: 0.2809160350321578
Epoch: 7, Train Loss: 0.2642690322046475
Epoch: 8, Train Loss: 0.24961276958473594
Epoch: 9, Train Loss: 0.23696492478144082
Epoch: 10, Train Loss: 0.22564292057239121
Epoch: 11, Train Loss: 0.21680739559859538
Epoch: 12, Train Loss: 0.20834367523033967
Epoch: 13, Train Loss: 0.1991484468255499
Epoch: 14, Train Loss: 0.19302650821433245
Epoch: 15, Train Loss: 0.18631490079866495
Epoch: 16, Train Loss: 0.180048742462054
Epoch: 17, Train Loss: 0.17341251395720306
Epoch: 18, Train Loss: 0.16781994960516142
Epoch: 19, Train Loss: 0.16345687049349325


In [32]:
# Lastly, we can set up an evaluation loop to test model performance.

classifier.eval()
test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in test_loader:
        data = data.view(-1, 1, 28, 28)
        output = classifier(data)
        test_loss += loss_function(output, target).item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader)
accuracy = 100.0 * correct / len(test_loader.dataset)
print("Test Loss: {:.4f}, Test Accuracy: {:.2f}%".format(test_loss, accuracy))

# model_weights: 91.64 test accuracy
# model_weights2: 

Test Loss: 0.2511, Test Accuracy: 92.08%


In [42]:
# Running inference on the test set with the saved model weights. 
# *** Remember to add one back to the labels in order to get the correct final outputs ***

test_filename = "features-test.csv"
test_data = pd.read_csv(test_filename)

def parse_test_data(data):
    X = test_data.iloc[:, 1:].values / 255 
    ids = test_data.iloc[:, 0].values
    return torch.tensor(X, dtype=torch.float32), ids

test_features, sample_ids = parse_test_data(test_data)

# # Load the trained model
# model = CharNet()
# model.load_state_dict(torch.load("model_weights.pth"))
classifier.eval()

# Run inference on the test set
with torch.no_grad():
    test_features = test_features.view(-1, 1, 28, 28)
    outputs = classifier(test_features)
    _, predicted_labels = torch.max(outputs, 1)
    predicted_labels += 1  

# Store the results
results = pd.DataFrame({"sample-id": sample_ids, "letter_idx": predicted_labels.numpy()})
results.to_csv("submission.csv", index=False)

